<a href="https://colab.research.google.com/github/trchudley/GEOG2462/blob/main/Short_Scripts/Week_1_Filter_by_AOI_coverage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a target="_blank" href="https://colab.research.google.com/github/trchudley/GEOG2462/blob/main/Short_Scripts/Week_1_Download_Single_Scene.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Filtering by coverage of your AOI

## Logging in to Google Earth Engine

Ensure the project name is your own, created upon registration with GEE. You can easily register one at the [following link](https://code.earthengine.google.com/register) - just make sure to select `Unpaid Usage` > `Academia & Research`).

In [1]:
import ee
import geemap
import time

ee.Authenticate()  # Trigger the authentication flow.
ee.Initialize(project='ee-trchudley')    # Change to your own default project name.

## Define editable variables

This is the only cell you will need to edit in this notebook.

In [7]:

# Define search parameters
latitude = 70.405   # Degrees of latitude
longitude = -50.519  # Degrees of longitude
size = 10000  # Size of AOI, in metres
region_name = 'storeglacier'  # AOI name, for filename construction

# Define search range, within which the least cloudy image will be found
date_start = '2023-05-01'
date_end = '2023-09-30'

# Google Drive export folder
folder = 'scires_project_2A'


# Searching for data without the filter

In [8]:
# Get search region geometry
point = ee.Geometry.Point(longitude, latitude)  # Create a point
region = point.buffer(size/2).bounds()  # Buffer the point to a 2D shape

# Get Landsat 8 image collection
landsat8_collection = ee.ImageCollection("LANDSAT/LC08/C02/T1_TOA")

# Filter to desired region and date bounds
landsat8_collection = landsat8_collection.filterBounds(region)
landsat8_collection = landsat8_collection.filterDate(date_start, date_end)

# landsat8_collection

# Get least cloudy image and clip to search region
image = landsat8_collection.sort('CLOUD_COVER').first()
image = image.clip(region)

image

Sanity check image:

In [9]:
Map = geemap.Map()  # Create empty map

max_reflectance = 1.00 # Set the upper limit of reflectance to visualise.
                       # Play with this value (between 0-1) to see what it
                       # does. It will need to be higher for snowy/icy
                       # scenes

visParams = {'bands': ['B4', 'B3', 'B2'], 'max': max_reflectance}
Map.addLayer(image, visParams, 'Colour Composite Image')

Map.centerObject(region, zoom=12)
Map

Map(center=[70.40499188079039, -50.518755412149005], controls=(WidgetControl(options=['position', 'transparent…

## Employing a simple presence/absence filter

# Download the image

In [ ]:
# Get the data of the image from the metadata
date_string = image.get('DATE_ACQUIRED').getInfo()

# Now we will construct the filename automatically
filename = region_name + '_' + date_string + '_image'

# Visualise for testing
print("The image will be saved to your Google Drive at:\n" + folder + '/' + filename + '.tif')

# Export the image, specifying scale and region.
task = ee.batch.Export.image.toDrive(**{
    'image': image.select(['B4', 'B3', 'B2', 'B5', 'B6']),
    'description': filename,
    'folder': folder,
    'scale': 30,
    'region': region.getInfo()['coordinates']
})
task.start()

while task.active():
  print('Task processing ongoing... (id: {}).'.format(task.id))
  time.sleep(5)

print('Finished processing. Image is exported to your Drive.')